# Torchserve

## Installing

In [2]:
! pip install torchserve torch-model-archiver torch-workflow-archiver
# or for conda environments
! conda install -c pytorch torchserve torch-model-archiver torch-workflow-archiver

After proper installation you should be able to call torcheserve and its utils from console like `torchserve ...`

In [5]:
! mkdir torchserve-examples
! git clone https://github.com/pytorch/serve.git torchserve-examples

Cloning into 'torchserve-examples'...
remote: Enumerating objects: 31558, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 31558 (delta 150), reused 217 (delta 138), pack-reused 31316
Receiving objects: 100% (31558/31558), 57.86 MiB | 19.49 MiB/s, done.
Resolving deltas: 100% (18885/18885), done.



## Model Packaging

In [6]:
! mkdir Transformer_model
! mv 2_src/setup_config.json torchserve-examples/examples/Huggingface_Transformers/setup_config.json

In [10]:
# Download the model according to provided config
! python torchserve-examples/examples/Huggingface_Transformers/Download_Transformer_models.py

Traceback (most recent call last):
  File "/Users/vdabravolski/repos/Accelerate-Deep-Learning-Workloads-with-Amazon-SageMaker/chapter9/torchserve-examples/examples/Huggingface_Transformers/Download_Transformer_models.py", line 1, in <module>
    import transformers
ModuleNotFoundError: No module named 'transformers'


In [14]:
! torch-model-archiver --model-name BERTQA --version 1.0 --serialized-file Transformer_model/pytorch_model.bin --handler torchserve-examples/examples/Huggingface_Transformers/Transformer_handler_generalized.py --extra-files "Transformer_model/config.json,./torchserve-examples/examples/Huggingface_Transformers/setup_config.json"

# This produces BERTQA.mar archive

In [9]:

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
#role = get_execution_role()  # TODO: replace it
role="arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230" # TODO: this has to be replaced

bucket = sagemaker_session.default_bucket()
prefix = 'torchserve'
s3_path = 's3://{}/{}'.format(bucket, prefix)


In [10]:

model_data = sagemaker_session.upload_data('distilbert-base-uncased-distilled-squad',
                                           bucket,
                                           os.path.join(prefix, 'model-artifacts'))

In [11]:
model_data

's3://sagemaker-us-east-1-941656036254/torchserve/model-artifacts'

In [1]:
model_data = "s3://sagemaker-us-east-1-941656036254/torchserve/model-artifacts"

# Local Mode

In [2]:
import boto3
from sagemaker.local import LocalSession

sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}
account = boto3.client('sts').get_caller_identity().get('Account')
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230"

/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/local_session.py:562: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.config = yaml.load(open(sagemaker_config_file, "r"))


In [3]:
import os
import subprocess

instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
except:
    pass

print("Instance type = " + instance_type)

Instance type = local


# Create Inference script

In [25]:
! pygmentize 2_src/pipeline-predictor.py

import torch
import os
import json
from transformers import pipeline


JSON_CONTENT_TYPE = "application/json"


def model_fn(model_dir):
    print(f"!!!!!!!!! loading model from {model_dir}")
    model_pipeline = pipeline(
        "question-answering",
        model=os.path.join(model_dir, "distilbert-base-uncased-distilled-squad"),
    )
    return model_pipeline


def input_fn(serialized_input_data, content_type=JSON_CONTENT_TYPE):
    print("****************")
    print(content_type)
    if content_type == JSON_CONTENT_TYPE:
        # input_data = json.loads(serialized_input_data)
        print("inside deser if")
        print(type(serialized_input_data))
        input_data = json.loads(serialized_input_data)
        print(f"!!!!!!!!! deser input data {input_data}")
        return input_data
    else:
        Exception("Requested unsupported ContentType in Accept: " + content_type)


def predict_fn(input_data, model_pipeline):
    print("Inside predict_fn")
    print(input_data)
   

# Packaging model artifacts

In [ ]:
# Download artifacts for Q&A model

! mkdir distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/pytorch_model.bin -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer.json -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/tokenizer_config.json -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/vocab.txt -P distilbert-base-uncased-distilled-squad
! wget https://huggingface.co/distilbert-base-uncased-distilled-squad/resolve/main/config.json -P distilbert-base-uncased-distilled-squad

Then package it into TAR archive. 

In [10]:
!tar -C "$PWD" -czf distilbert-base-uncased-distilled-squad.tar.gz  distilbert-base-uncased-distilled-squad/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
#role = get_execution_role()  # TODO: replace it
role="arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230" # TODO: this has to be replaced

bucket = sagemaker_session.default_bucket()
prefix = 'torchserve'
s3_path = 's3://{}/{}'.format(bucket, prefix)


In [28]:

model_data = sagemaker_session.upload_data('distilbert-base-uncased-distilled-squad',
                                           bucket,
                                           os.path.join(prefix, 'model-artifacts'))

06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:12:44,116 [INFO ] pool-2-thread-1 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057964
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:12:44,117 [INFO ] pool-2-thread-1 TS_METRICS - DiskAvailable.Gigabytes:136.1949005126953|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057964
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:12:44,117 [INFO ] pool-2-thread-1 TS_METRICS - DiskUsage.Gigabytes:147.5674705505371|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057964
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:12:44,117 [INFO ] pool-2-thread-1 TS_METRICS - DiskUtilization.Percent:52.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057964
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:12:44,117 [INFO ] pool-2-thread-1 TS_METRICS - MemoryAvailable.Megabytes:2387.6875|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057964
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:12:44,117 [INFO ] pool-2-thread-1 TS_METRICS - MemoryUsed.Megabytes:2954.

In [32]:
#model_data = "s3://sagemaker-us-east-1-941656036254/torchserve/model/distilbert-base-uncased-distilled-squad.tar.gz"

# Deployed Locally

In [4]:
import boto3
from sagemaker.local import LocalSession

sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/local_session.py:562: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.config = yaml.load(open(sagemaker_config_file, "r"))


In [5]:
import os
import subprocess

instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
except:
    pass

print("Instance type = " + instance_type)

Instance type = local


In [23]:
from sagemaker.pytorch import PyTorchModel

env = {    "SAGEMAKER_TS_BATCH_SIZE": "3",
    "SAGEMAKER_TS_MAX_BATCH_DELAY": "100000"
    }


# Note: You can update the 'torchserve-predictor.py' file as needed according to the model you want to use (ie BERT) 
model = PyTorchModel(model_data=model_data,
                   role=role, 
                   entry_point='pipeline-predictor.py',
                   source_dir='2_src',
                   framework_version='1.9.0',
                   py_version='py38',
                   env=env,
                   sagemaker_session=sagemaker_local_session)

In [24]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

local_predictor = model.deploy(initial_instance_count=1, instance_type=instance_type, serializer=JSONSerializer(), deserializer=JSONDeserializer())

06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:55:44,911 [INFO ] pool-2-thread-2 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660056944
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:55:44,911 [INFO ] pool-2-thread-2 TS_METRICS - DiskAvailable.Gigabytes:136.1950569152832|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660056944
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:55:44,912 [INFO ] pool-2-thread-2 TS_METRICS - DiskUsage.Gigabytes:147.56731414794922|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660056944
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:55:44,912 [INFO ] pool-2-thread-2 TS_METRICS - DiskUtilization.Percent:52.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660056944
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:55:44,912 [INFO ] pool-2-thread-2 TS_METRICS - MemoryAvailable.Megabytes:2359.63671875|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660056944
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:55:44,912 [INFO ] pool-2-thread-2 TS_METRICS - MemoryUsed.Megabytes:

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/zm/7yczzhb529xff1v0_ps_8k4h0000gp/T/tmpz0veqmxo/docker-compose.yaml

06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:56:44,888 [INFO ] pool-2-thread-2 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057004
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:56:44,889 [INFO ] pool-2-thread-2 TS_METRICS - DiskAvailable.Gigabytes:136.19493865966797|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057004
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:56:44,889 [INFO ] pool-2-thread-2 TS_METRICS - DiskUsage.Gigabytes:147.56743240356445|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057004
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:56:44,890 [INFO ] pool-2-thread-2 TS_METRICS - DiskUtilization.Percent:52.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057004
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:56:44,890 [INFO ] pool-2-thread-2 TS_METRICS - MemoryAvailable.Megabytes:2375.84375|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660057004
06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:56:44,891 [INFO ] pool-2-thread-2 TS_METRICS - MemoryUsed.Megabytes:29

In [22]:
import json

context = r"""
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.
"""

question="What kind of forest is Amazon?"


data = {"question":question, "context":context}
print(data)
local_predictor.predict(data)


{'question': 'What kind of forest is Amazon?', 'context': '\nThe Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet\'s remaining rainforests, and comprises the largest and most biodiverse

{'score': 0.4394713044166565,
 'start': 238,
 'end': 253,
 'answer': 'moist broadleaf'}

06s8ui5cf3-algo-1-97ynt | 2022-08-09 14:47:43,128 [INFO ] W-9001-model_1-stdout MODEL_LOG - !!!!!!!!! deser input data {'question': 'What kind of forest is Amazon?', 'context': '\nThe Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names

In [33]:
from sagemaker.pytorch import PyTorchModel

env = {    "SAGEMAKER_TS_BATCH_SIZE": "3",
    "SAGEMAKER_TS_MAX_BATCH_DELAY": "100000"
    }


# Note: You can update the 'torchserve-predictor.py' file as needed according to the model you want to use (ie BERT) 
model = PyTorchModel(model_data=model_data,
                   role=role, 
                   entry_point='pipeline-predictor.py',
                   source_dir='2_src',
                   framework_version='1.9.0',
                   py_version='py38',
                   env=env,
                   sagemaker_session=sagemaker_session)

06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:20:43,767 [INFO ] pool-2-thread-2 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058443
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:20:43,768 [INFO ] pool-2-thread-2 TS_METRICS - DiskAvailable.Gigabytes:136.19488143920898|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058443
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:20:43,768 [INFO ] pool-2-thread-2 TS_METRICS - DiskUsage.Gigabytes:147.56748962402344|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058443
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:20:43,768 [INFO ] pool-2-thread-2 TS_METRICS - DiskUtilization.Percent:52.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058443
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:20:43,768 [INFO ] pool-2-thread-2 TS_METRICS - MemoryAvailable.Megabytes:2386.640625|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058443
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:20:43,768 [INFO ] pool-2-thread-2 TS_METRICS - MemoryUsed.Megabytes:2

In [34]:
remote_predictor = model.deploy(initial_instance_count=1, instance_type="ml.g4dn.4xlarge", serializer=JSONSerializer(), deserializer=JSONDeserializer())

-06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:21:43,730 [INFO ] pool-2-thread-1 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058503
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:21:43,731 [INFO ] pool-2-thread-1 TS_METRICS - DiskAvailable.Gigabytes:136.19488143920898|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058503
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:21:43,731 [INFO ] pool-2-thread-1 TS_METRICS - DiskUsage.Gigabytes:147.56748962402344|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058503
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:21:43,731 [INFO ] pool-2-thread-1 TS_METRICS - DiskUtilization.Percent:52.0|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058503
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:21:43,731 [INFO ] pool-2-thread-1 TS_METRICS - MemoryAvailable.Megabytes:2386.42578125|#Level:Host|#hostname:d17bd63b9d77,timestamp:1660058503
06s8ui5cf3-algo-1-97ynt | 2022-08-09 15:21:43,731 [INFO ] pool-2-thread-1 TS_METRICS - MemoryUsed.Megabyte

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/vdabravolski/miniconda/envs/sagemaker2/lib/python3.9/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/private/var/folders/zm/7yczzhb529xff1v0_ps_8k4h0000gp/T/tmpzizybp1m/docker-compose.ya

--------!

In [35]:
remote_predictor.predict(data)

{'score': 0.4394714832305908,
 'start': 238,
 'end': 253,
 'answer': 'moist broadleaf'}

In [ ]:
from time import perf_counter

def timer(f,*args):   
    
    start = perf_counter()
    f(*args)
    return (1000 * (perf_counter() - start))


stats = np.array([timer(predict_f, local_predictor) for _ in range(10)])
print(stats)

for i in range(100):
    local_predictor.predict(data)

In [11]:
local_predictor.endpoint_name

'pytorch-inference-2022-08-09-03-08-44-728'